In [1]:
import pandas as pd
import geopandas as gpd
import requests
import altair as alt
import tqdm
import numpy as np

pd.set_option('display.max_columns', None)

# Read In Voter Registration Data

In [2]:
def read_reg_voter_lists(folder, num_files):
    data = []
    for i in tqdm.trange(1, num_files+1):
        data.append(pd.read_csv(path+folder+file_base+str(i)+".txt", engine="python"))

    raw = pd.concat(data)
    raw = raw.reset_index()
    return raw

In [3]:
path = "../data/raw/registration/"
file_base = "Registered_Voters_List_Part"

In [4]:
folder = "2018_Nov/"
raw_2018 = read_reg_voter_lists(folder, 8)

100%|██████████| 8/8 [02:14<00:00, 16.82s/it]


In [5]:
folder = "2016_Dec/"
raw_2016 = read_reg_voter_lists(folder, 8)

100%|██████████| 8/8 [02:14<00:00, 16.76s/it]


In [6]:
folder = "2014_Dec/"
file = open(path+folder+file_base+str(1)+".txt", 'r') 
lines = file.readlines()
header = lines[0].strip().split('|')

In [7]:
dats = []
for j in tqdm.trange(1,12):
    file = open(path+folder+file_base+str(j)+".txt", 'r') 
    lines = file.readlines()

    data = np.zeros((len(lines), len(header))).astype('object')

    for i, line in enumerate(lines[1:]):
        parsed_line = line.strip().split('|')
        if len(parsed_line) == len(header):
            data[i,:] = parsed_line
        else:
            print(parsed_line)
    
    dat = pd.DataFrame(data=data, columns=header)
    dats.append(dat)

100%|██████████| 11/11 [00:55<00:00,  5.08s/it]


In [8]:
raw_2014 = pd.concat(dats)
raw_2014 = raw_2014.reset_index()
raw_2014.head()

,index,VOTER_ID,COUNTY_CODE,COUNTY,LAST_NAME,FIRST_NAME,MIDDLE_NAME,NAME_SUFFIX,VOTER_NAME,STATUS_CODE,PRECINCT_NAME,ADDRESS_LIBRARY_ID,HOUSE_NUM,HOUSE_SUFFIX,PRE_DIR,STREET_NAME,STREET_TYPE,POST_DIR,UNIT_TYPE,UNIT_NUM,ADDRESS_NON_STD,RESIDENTIAL_ADDRESS,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP_CODE,RESIDENTIAL_ZIP_PLUS,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,STATUS_REASON,BIRTH_YEAR,GENDER,PRECINCT,SPLIT,VOTER_STATUS_ID,PARTY,PARTY_AFFILIATION_DATE,PHONE_NUM,MAIL_ADDR1,MAIL_ADDR2,MAIL_ADDR3,MAILING_CITY,MAILING_STATE,MAILING_ZIP_CODE,MAILING_ZIP_PLUS,MAILING_COUNTRY,SPL_ID,PERMANENT_MAIL_IN_VOTER,CONGRESSIONAL,STATE_SENATE,STATE_HOUSE,ID_REQUIRED
0,0,544384,47,Park,SAVAGE,CHRISTINE,LEE,,"SAVAGE, CHRISTINE LEE",A,2026047013,4927,206,,,SPRING,CIR,,,,,206 SPRING CIR,PINE,CO,80470,7964,06/10/2008,07/30/1984,Active,,1947,Female,2026047013,013.090,1,REP,07/30/1984,,206 SPRING CIR,,,PINE,CO,80470,7964,UNITED STATES OF AMERICA,690,No,CONGRESSIONAL 2,STATE SENATE 2,STATE HOUSE 60,N
1,1,545252,47,Park,KLEIN,LINDA,REZNICEK,,"KLEIN, LINDA REZNICEK",A,2026047013,4970,89,,,JONES CREEK,CIR,,,,,89 JONES CREEK CIR,PINE,CO,80470,9676,04/23/2012,09/06/1984,Active,,1951,Female,2026047013,013.090,1,REP,04/22/2012,3038386638,89 JONES CREEK CIR,,,PINE,CO,80470,9676,UNITED STATES OF AMERICA,690,No,CONGRESSIONAL 2,STATE SENATE 2,STATE HOUSE 60,N
2,2,600021467,21,El Paso,WATSON-AUSENCIO,PAUL,JOHN,,"WATSON-AUSENCIO, PAUL JOHN",A,5091921444,21320,7535,,,GALLARD,HTS,,,,,7535 GALLARD HTS,PEYTON,CO,80831,4504,11/22/2013,10/09/2007,Active,,1983,Male,5091921444,444.SB9.4,1,REP,11/22/2013,719-360-9578,,,,,,,,,6633,No,CONGRESSIONAL 5,STATE SENATE 9,STATE HOUSE 19,N
3,3,600279824,21,El Paso,STAMEY,LORI,KAY,,"STAMEY, LORI KAY",A,5121521414,29547,6675,,,PAWNEE,CIR,,,,,6675 PAWNEE CIR,COLO SPRINGS,CO,80915,2629,06/23/2010,06/23/2010,Active,,1960,Female,5121521414,414.SCG,1,REP,06/23/2010,,1347 BATES DR,,,COLO SPRINGS,CO,80909,3142,UNITED STATES OF AMERICA,6470,No,CONGRESSIONAL 5,STATE SENATE 12,STATE HOUSE 15,N
4,4,600752295,21,El Paso,REISINGER,CURTIS,ART ARNOLD,,"REISINGER, CURTIS ART ARNOLD",A,5111821177,13478,423,1/2,E,BIJOU,ST,,,,,423 1/2 E BIJOU ST,COLO SPRINGS,CO,80903,,11/04/2014,03/22/2013,Active,Failed to Vote,1991,Male,5111821177,177.FBC,1,REP,03/22/2013,,515 N TEJON ST,,,COLO SPRINGS,CO,80903,1145,UNITED STATES OF AMERICA,7268,No,CONGRESSIONAL 5,STATE SENATE 11,STATE HOUSE 18,N


In [9]:
raw_2014.isnull().sum().sum()

0

# Deal With Bad Entries

## 2018 

In [10]:
bad_dates_2018 = {
    1379196: '04/12/2018',
    2546940: '04/27/2018',
    2561258: '08/27/2018',
    3132565: '08/31/2018',
    3846569: '09/07/2018',
    3871991: '10/04/2018',
}

for id_val in bad_dates_2018:
    raw_2018.loc[id_val,'PARTY_AFFILIATION_DATE'] = bad_dates_2018[id_val]

## 2014

In [24]:
sel = raw_2014.EFFECTIVE_DATE.str.contains('0.0', regex=False, na=True)
idxs = raw_2014[sel].index
raw_2014_mod = raw_2014.drop(idxs, axis=0)

# Convert to better types

In [12]:
def convert_types(df, to_drop, to_datetime, to_category, to_str):
    reg_voters = df.drop(columns=to_drop)
    for col in to_datetime:
        reg_voters[col] = pd.to_datetime(reg_voters[col], infer_datetime_format=True)
    for col in to_str+to_category:
        reg_voters[col] = reg_voters[col].astype('str')
    for col in to_category:
        reg_voters[col] = reg_voters[col].astype('category')
    return reg_voters

In [13]:
to_drop = ["ADDRESS_LIBRARY_ID", "HOUSE_NUM", "HOUSE_SUFFIX", "PRE_DIR", 
           "STREET_NAME", "STREET_TYPE", "POST_DIR", "UNIT_TYPE", "SPLIT",
           "PHONE_NUM", "MAIL_ADDR1", "MAIL_ADDR2", "MAIL_ADDR3",
           "MAILING_CITY", "MAILING_STATE", "MAILING_ZIP_CODE",
           "MAILING_ZIP_PLUS", "MAILING_COUNTRY", "SPL_ID", "ID_REQUIRED", 
           "PARTY_AFFILIATION_DATE", "PREFERENCE"]
to_datetime = ["EFFECTIVE_DATE", "REGISTRATION_DATE"]
to_category = ["COUNTY_CODE", "COUNTY", "NAME_SUFFIX", "STATUS_CODE",
               "PRECINCT_NAME", "RESIDENTIAL_STATE", "RESIDENTIAL_ZIP_CODE",
               "STATUS", "STATUS_REASON", "BIRTH_YEAR", "GENDER", "PRECINCT",
               "VOTER_STATUS_ID", "PARTY", "PERMANENT_MAIL_IN_VOTER",
               "CONGRESSIONAL", "STATE_SENATE", "STATE_HOUSE"]
to_str = ["VOTER_ID", "LAST_NAME", "FIRST_NAME", "MIDDLE_NAME", "RESIDENTIAL_ADDRESS",
          "RESIDENTIAL_CITY", "RESIDENTIAL_ZIP_PLUS", "VOTER_NAME"]

In [14]:
reg_voters_2018 = convert_types(raw_2018, to_drop, to_datetime, to_category, to_str)

In [15]:
to_drop_2016 = to_drop.copy() + ["ADDRESS_NON_STD"]
to_drop_2016.remove("PREFERENCE")
reg_voters_2016 = convert_types(raw_2016, to_drop_2016, to_datetime, to_category, to_str)

In [25]:
for col in raw_2014_mod.columns[1:]:
    raw_2014_mod[col] = raw_2014_mod[col].astype('str')
    
reg_voters_2014 = convert_types(raw_2014_mod, to_drop_2016, to_datetime, to_category, to_str)

In [26]:
reg_voters_years = [reg_voters_2018, reg_voters_2016, reg_voters_2014]
column_names = [rv.columns.values for rv in reg_voters_years]

for i, (j,k) in enumerate(zip(column_names[0], column_names[2])):
    print(i, j, k, j==k)

0 index index True
1 VOTER_ID VOTER_ID True
2 COUNTY_CODE COUNTY_CODE True
3 COUNTY COUNTY True
4 LAST_NAME LAST_NAME True
5 FIRST_NAME FIRST_NAME True
6 MIDDLE_NAME MIDDLE_NAME True
7 NAME_SUFFIX NAME_SUFFIX True
8 VOTER_NAME VOTER_NAME True
9 STATUS_CODE STATUS_CODE True
10 PRECINCT_NAME PRECINCT_NAME True
11 UNIT_NUM UNIT_NUM True
12 RESIDENTIAL_ADDRESS RESIDENTIAL_ADDRESS True
13 RESIDENTIAL_CITY RESIDENTIAL_CITY True
14 RESIDENTIAL_STATE RESIDENTIAL_STATE True
15 RESIDENTIAL_ZIP_CODE RESIDENTIAL_ZIP_CODE True
16 RESIDENTIAL_ZIP_PLUS RESIDENTIAL_ZIP_PLUS True
17 EFFECTIVE_DATE EFFECTIVE_DATE True
18 REGISTRATION_DATE REGISTRATION_DATE True
19 STATUS STATUS True
20 STATUS_REASON STATUS_REASON True
21 BIRTH_YEAR BIRTH_YEAR True
22 GENDER GENDER True
23 PRECINCT PRECINCT True
24 VOTER_STATUS_ID VOTER_STATUS_ID True
25 PARTY PARTY True
26 PERMANENT_MAIL_IN_VOTER PERMANENT_MAIL_IN_VOTER True
27 CONGRESSIONAL CONGRESSIONAL True
28 STATE_SENATE STATE_SENATE True
29 STATE_HOUSE STATE_HOUSE

# Save

In [18]:
reg_voters_2018.to_feather("../data/processed/registration/trimmed_2018.feather")

In [19]:
reg_voters_2016.to_feather("../data/processed/registration/trimmed_2016.feather")

In [28]:
reg_voters_2014 = reg_voters_2014.reset_index()
reg_voters_2014.to_feather("../data/processed/registration/trimmed_2014.feather")

# Load

In [29]:
reg_voters_2018 = pd.read_feather("../data/processed/registration/trimmed_2018.feather")

In [30]:
reg_voters_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3922281 entries, 0 to 3922280
Data columns (total 30 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   index                    int64         
 1   VOTER_ID                 object        
 2   COUNTY_CODE              category      
 3   COUNTY                   category      
 4   LAST_NAME                object        
 5   FIRST_NAME               object        
 6   MIDDLE_NAME              object        
 7   NAME_SUFFIX              category      
 8   VOTER_NAME               object        
 9   STATUS_CODE              category      
 10  PRECINCT_NAME            category      
 11  UNIT_NUM                 object        
 12  RESIDENTIAL_ADDRESS      object        
 13  RESIDENTIAL_CITY         object        
 14  RESIDENTIAL_STATE        category      
 15  RESIDENTIAL_ZIP_CODE     category      
 16  RESIDENTIAL_ZIP_PLUS     object        
 17  EFFECTIVE_DATE           da

In [31]:
reg_voters_2016 = pd.read_feather("../data/processed/registration/trimmed_2016.feather")

In [34]:
reg_voters_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3836648 entries, 0 to 3836647
Data columns (total 30 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   index                    int64         
 1   VOTER_ID                 object        
 2   COUNTY_CODE              category      
 3   COUNTY                   category      
 4   LAST_NAME                object        
 5   FIRST_NAME               object        
 6   MIDDLE_NAME              object        
 7   NAME_SUFFIX              category      
 8   VOTER_NAME               object        
 9   STATUS_CODE              category      
 10  PRECINCT_NAME            category      
 11  UNIT_NUM                 object        
 12  RESIDENTIAL_ADDRESS      object        
 13  RESIDENTIAL_CITY         object        
 14  RESIDENTIAL_STATE        category      
 15  RESIDENTIAL_ZIP_CODE     category      
 16  RESIDENTIAL_ZIP_PLUS     object        
 17  EFFECTIVE_DATE           da

In [39]:
reg_voters_2014 = pd.read_feather("../data/processed/registration/trimmed_2014.feather")
reg_voters_2014 = reg_voters_2014.drop(columns=["level_0"])

In [40]:
reg_voters_2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644865 entries, 0 to 3644864
Data columns (total 30 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   index                    int64         
 1   VOTER_ID                 object        
 2   COUNTY_CODE              category      
 3   COUNTY                   category      
 4   LAST_NAME                object        
 5   FIRST_NAME               object        
 6   MIDDLE_NAME              object        
 7   NAME_SUFFIX              category      
 8   VOTER_NAME               object        
 9   STATUS_CODE              category      
 10  PRECINCT_NAME            category      
 11  UNIT_NUM                 object        
 12  RESIDENTIAL_ADDRESS      object        
 13  RESIDENTIAL_CITY         object        
 14  RESIDENTIAL_STATE        category      
 15  RESIDENTIAL_ZIP_CODE     category      
 16  RESIDENTIAL_ZIP_PLUS     object        
 17  EFFECTIVE_DATE           da

# Extract Meaningful Features

What are the important features that I want to extract out of this dataset to look at the precinct level?
  * (6) count active/inactive by gender
  * (21) count gender by age 18-24, 25-34, 35-44, ... , 75+
  * (3) average length of registration by gender
  * (24) party affiliation by gender
  
54 variables. Let's do it

In [41]:
def extract_active_inactive(df):
    df2 = df.copy()
    df2.STATUS_CODE = df2.STATUS_CODE.cat.rename_categories({'A': 'active', 'I': 'inactive'})
    count_active_inactive_by_gender = df2.groupby(["PRECINCT", "GENDER", "STATUS_CODE"]).index.count()
    count_active_inactive_by_gender = count_active_inactive_by_gender.unstack().unstack()
    return count_active_inactive_by_gender

In [105]:
def extract_age_ranges(df):
    age_ranges = np.array([18,29,30,49,50,64,65,200])
    #print(1, age_ranges)
    age_range_labels = ["18-29", "30-49", "50-64", "over 65"]
    #print(2, age_range_labels)
    age_range_labels = np.array(age_range_labels*3).reshape((3,4)).T.reshape((12))
    #print(3, age_range_labels)
    years = 2018 - age_ranges
    #print(4, years)
    year_ranges = [(j, i) for (i, j) in years.reshape((4,2))]
    #print(5, year_ranges)
    bins = pd.IntervalIndex.from_tuples(year_ranges)
    #print(6, bins)
    #print(6.1, df.BIRTH_YEAR.to_list())
    #print(6.2, bins)
    #print(6.3, age_range_labels)
    bins = pd.cut(df.BIRTH_YEAR.astype(int).to_list(), bins, labels=age_range_labels)
    #print(7, bins)

    count_gender_by_age = df.groupby(["PRECINCT", "GENDER", bins]).index.count()
    count_gender_by_age = count_gender_by_age.unstack().unstack()

    new_labels = [(k,j) for (i, j), k in zip(count_gender_by_age.columns,age_range_labels)]
    count_gender_by_age.columns = pd.MultiIndex.from_tuples(new_labels, names=["AGE", "GENDER"])
    return count_gender_by_age

In [89]:
test = extract_age_ranges(reg_voters_2018.head())

In [90]:
test.loc['2026047010']

AGE      GENDER 
18_29    Female     0
         Male       0
         Unknown    0
30_49    Female     0
         Male       0
         Unknown    0
50_64    Female     0
         Male       0
         Unknown    0
65 plus  Female     1
         Male       0
         Unknown    0
Name: 2026047010, dtype: int64

In [60]:
reg_voters_2018.head()

,index,VOTER_ID,COUNTY_CODE,COUNTY,LAST_NAME,FIRST_NAME,MIDDLE_NAME,NAME_SUFFIX,VOTER_NAME,STATUS_CODE,PRECINCT_NAME,UNIT_NUM,RESIDENTIAL_ADDRESS,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP_CODE,RESIDENTIAL_ZIP_PLUS,EFFECTIVE_DATE,REGISTRATION_DATE,STATUS,STATUS_REASON,BIRTH_YEAR,GENDER,PRECINCT,VOTER_STATUS_ID,PARTY,PERMANENT_MAIL_IN_VOTER,CONGRESSIONAL,STATE_SENATE,STATE_HOUSE
0,0,544769,47,Park,IVERSON,JOANN,ELAINE,nan,"IVERSON, JOANN ELAINE",A,2026047010,None,44 BANDIT LN,BAILEY,CO,80421,1344.0,1988-10-14,1988-10-14,Active,nan,1931,Female,2026047010,1,DEM,No,Congressional 2,State Senate 2,State House 60
1,1,544770,47,Park,WILLIAMS,WILLIAM,J,nan,"WILLIAMS, WILLIAM J",A,5026047006,None,555 CASTLE MOUNTAIN DR,GUFFEY,CO,80820,nan,2008-06-03,1977-10-07,Active,nan,1941,Male,5026047006,1,REP,No,Congressional 5,State Senate 2,State House 60
2,2,544772,47,Park,SPRINGER,GINGER,FAYE,nan,"SPRINGER, GINGER FAYE",A,5026047007,None,1444 NAVAJO TRL,HARTSEL,CO,80449,nan,2008-06-12,1982-08-06,Active,nan,1946,Female,5026047007,1,REP,No,Congressional 5,State Senate 2,State House 60
3,3,544775,47,Park,GEBAUER,LARRY,LAVERN,nan,"GEBAUER, LARRY LAVERN",A,2026047013,None,789 WISP CREEK DR,BAILEY,CO,80421,nan,2013-05-16,1980-07-12,Active,nan,1936,Male,2026047013,1,UAF,No,Congressional 2,State Senate 2,State House 60
4,4,544776,47,Park,SCHROEDER,GLENDA,KAE,nan,"SCHROEDER, GLENDA KAE",A,2026047013,None,97 NOVA RD,PINE,CO,80470,5102.0,2008-06-02,1982-09-30,Active,nan,1940,Female,2026047013,1,UAF,No,Congressional 2,State Senate 2,State House 60


In [43]:
def extract_mean_registration_length(df):
    df2 = df.copy()
    df2.REGISTRATION_DATE = (pd.to_datetime('11/1/2018') - df2.REGISTRATION_DATE).values.astype(np.int64)
    mean_reg_by_gender = df2.groupby(["PRECINCT", "GENDER"])["REGISTRATION_DATE"].mean()
    mean_reg_by_gender = pd.to_timedelta(mean_reg_by_gender, unit='ns')
    mean_reg_by_gender = mean_reg_by_gender.unstack()
    mean_reg_by_gender.columns = pd.MultiIndex.from_tuples([("mean_reg_time", "Female"),
                                                            ("mean_reg_time", "Male"),
                                                            ("mean_reg_time", "Unknown")])
    return mean_reg_by_gender

In [44]:
def extract_count_party(df):
    count_party_by_gender = df.groupby(["PRECINCT", "GENDER", "PARTY"]).index.count()
    count_party_by_gender = count_party_by_gender.unstack().unstack()
    return count_party_by_gender

In [106]:
def extract_voter_reg_vars(df):
    count_active_inactive_by_gender = extract_active_inactive(df)
    count_gender_by_age = extract_age_ranges(df)
    mean_reg_by_gender = extract_mean_registration_length(df)
    count_party_by_gender = extract_count_party(df)
    
    to_concat = [count_active_inactive_by_gender,
                 count_gender_by_age,
                 mean_reg_by_gender,
                 count_party_by_gender]

    for item in to_concat:
        item.columns = item.columns.to_list()
        
    reg_voter_data = pd.concat(to_concat, axis=1)
    reg_voter_data.columns = [gender.lower()+" rv_"+var.lower() for (var, gender) in reg_voter_data.columns]
    
    return reg_voter_data

In [107]:
reg_voter_data_2018 = extract_voter_reg_vars(reg_voters_2018)
reg_voter_data_2018.head()

,female rv_active,male rv_active,unknown rv_active,female rv_inactive,male rv_inactive,unknown rv_inactive,female rv_18-29,male rv_18-29,unknown rv_18-29,female rv_30-49,male rv_30-49,unknown rv_30-49,female rv_50-64,male rv_50-64,unknown rv_50-64,female rv_over 65,male rv_over 65,unknown rv_over 65,female rv_mean_reg_time,male rv_mean_reg_time,unknown rv_mean_reg_time,female rv_acn,male rv_acn,unknown rv_acn,female rv_apv,male rv_apv,unknown rv_apv,female rv_dem,male rv_dem,unknown rv_dem,female rv_grn,male rv_grn,unknown rv_grn,female rv_lbr,male rv_lbr,unknown rv_lbr,female rv_rep,male rv_rep,unknown rv_rep,female rv_uaf,male rv_uaf,unknown rv_uaf,female rv_uni,male rv_uni,unknown rv_uni
PRECINCT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1160116112,557,495,17,61,58,4,78,82,7,174,158,9,137,124,2,194,160,3,5176 days 19:04:04.660194,4772 days 14:53:09.873417,933 days 19:25:42.857142,2,4,0,0,0,0,249,179,4,3,0,1,10,5,1,139,141,7,215,224,8,0,0,0
1160116113,512,480,14,31,32,1,57,61,6,174,163,3,127,133,3,152,123,2,6725 days 02:36:27.845303,5915 days 04:49:41.250000,649 days 08:00:00,0,0,0,0,0,0,258,194,2,0,2,0,4,7,0,108,121,2,173,188,11,0,0,0
1160116114,611,570,18,82,79,6,128,135,12,224,193,7,145,149,3,157,136,2,5406 days 06:53:30.389610,5128 days 03:37:26.533127,489 days 20:00:00,1,3,0,0,0,0,322,211,7,0,0,0,5,10,0,131,147,1,232,278,16,2,0,0
1160116120,719,685,48,175,198,10,215,195,33,303,331,15,186,159,5,137,131,2,3140 days 23:42:16.912751,3089 days 02:03:56.466591,507 days 07:02:04.137931,1,5,0,0,0,0,376,276,13,6,6,0,17,15,3,151,171,4,342,409,38,1,1,0
1160116122,591,594,17,56,48,2,92,113,7,185,190,2,163,150,5,184,168,5,4942 days 02:04:38.207109,4575 days 21:56:38.130841,777 days 03:47:22.105263,2,1,0,0,0,0,283,191,4,2,3,1,5,11,0,156,173,1,199,263,13,0,0,0


In [108]:
reg_voter_data_2016 = extract_voter_reg_vars(reg_voters_2016)
reg_voter_data_2016.head()

,female rv_active,male rv_active,unknown rv_active,female rv_inactive,male rv_inactive,unknown rv_inactive,female rv_18-29,male rv_18-29,unknown rv_18-29,female rv_30-49,male rv_30-49,unknown rv_30-49,female rv_50-64,male rv_50-64,unknown rv_50-64,female rv_over 65,male rv_over 65,unknown rv_over 65,female rv_mean_reg_time,male rv_mean_reg_time,unknown rv_mean_reg_time,female rv_acn,male rv_acn,unknown rv_acn,female rv_dem,male rv_dem,unknown rv_dem,female rv_grn,male rv_grn,unknown rv_grn,female rv_lbr,male rv_lbr,unknown rv_lbr,female rv_rep,male rv_rep,unknown rv_rep,female rv_uaf,male rv_uaf,unknown rv_uaf,female rv_uni,male rv_uni,unknown rv_uni
PRECINCT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1160116112,561,495,14,70,71,2,63,72,5,174,181,7,140,119,1,218,169,3,5790 days 12:14:50.015847,5382 days 21:04:27.137809,1297 days 06:00:00,2,5,0,258,193,3,3,1,1,6,4,1,147,144,4,215,219,7,0,0,0
1160116113,530,470,10,36,38,1,55,53,4,171,156,4,131,129,1,174,136,0,7110 days 09:12:05.088339,6510 days 23:00:28.346456,886 days 06:32:43.636363,0,1,0,263,202,2,2,1,0,6,4,0,122,126,2,173,174,7,0,0,0
1160116114,617,542,12,89,92,1,102,99,6,232,192,4,161,154,2,173,147,1,6226 days 12:53:01.869688,5836 days 13:03:35.772870,814 days 05:32:18.461538,1,4,0,337,218,2,1,2,0,4,10,0,153,148,2,209,252,9,1,0,0
1160116120,730,658,29,194,228,6,172,142,15,338,376,12,201,173,3,152,138,2,3740 days 08:46:45.194805,3755 days 02:05:08.803611,979 days 04:06:51.428571,3,2,0,380,286,9,7,6,0,18,18,1,174,196,3,342,378,22,0,0,0
1160116122,607,576,21,51,65,2,72,80,10,212,208,3,155,154,7,191,175,3,5481 days 14:55:04.559270,5259 days 05:39:13.198127,843 days 07:18:15.652173,3,2,0,302,205,7,2,6,1,6,3,0,165,184,1,180,241,14,0,0,0


In [109]:
reg_voter_data_2014 = extract_voter_reg_vars(reg_voters_2014)
reg_voter_data_2014.head()

,female rv_active,male rv_active,unknown rv_active,female rv_inactive,male rv_inactive,unknown rv_inactive,female rv_18-29,male rv_18-29,unknown rv_18-29,female rv_30-49,male rv_30-49,unknown rv_30-49,female rv_50-64,male rv_50-64,unknown rv_50-64,female rv_over 65,male rv_over 65,unknown rv_over 65,female rv_mean_reg_time,male rv_mean_reg_time,unknown rv_mean_reg_time,female rv_acn,male rv_acn,unknown rv_acn,female rv_dem,male rv_dem,unknown rv_dem,female rv_grn,male rv_grn,unknown rv_grn,female rv_lbr,male rv_lbr,unknown rv_lbr,female rv_rep,male rv_rep,unknown rv_rep,female rv_uaf,male rv_uaf,unknown rv_uaf,female rv_uni,male rv_uni,unknown rv_uni
PRECINCT,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1160116112,511,466,3,145,126,1,45,53,1,167,198,2,138,118,1,268,199,0,7183 days 03:54:52.682926,6348 days 06:07:17.837837,2653 days 00:00:00,4,4,0,263,202,0,2,2,0,5,8,1,164,167,1,218,209,2,0,0,0
1160116113,513,476,2,57,62,0,33,34,0,146,146,1,144,146,1,211,176,0,8357 days 11:52:25.263157,7512 days 07:37:41.710037,1836 days 00:00:00,1,2,0,261,198,0,1,1,0,4,2,0,137,155,1,166,180,1,0,0,0
1160116114,570,503,1,138,130,0,63,67,1,238,214,0,163,141,0,203,170,0,7393 days 22:24:24.406779,6929 days 03:36:06.824644,1458 days 00:00:00,1,4,0,329,227,0,0,1,0,7,6,0,165,155,0,206,240,1,0,0,0
1160116120,672,605,1,411,391,1,116,96,0,465,485,0,228,202,1,192,157,0,4723 days 16:14:37.562326,4699 days 23:42:39.036144,1951 days 12:00:00,0,3,0,438,316,1,6,4,0,9,11,0,204,244,0,426,418,1,0,0,0
1160116122,581,531,2,114,131,0,53,54,1,238,223,0,166,159,1,206,202,0,6376 days 19:14:04.316546,6160 days 10:06:53.293051,1481 days 12:00:00,3,2,0,286,198,1,2,2,0,3,4,0,195,213,0,206,243,1,0,0,0


# Save

In [110]:
reg_voter_data_2018.to_csv("../data/processed/registration/reg_voter_2018.csv")

In [111]:
reg_voter_data_2016.to_csv("../data/processed/registration/reg_voter_2016.csv")

In [112]:
reg_voter_data_2014.to_csv("../data/processed/registration/reg_voter_2014.csv")